In [ ]:
# Import all libraries and modules, define variables and objects needed to run the code

import pandas as pd
import numpy as np

# Load progress idicator tools
!pip install tqdm
from tqdm import tqdm

# For loading dataframes by open link to google drive, to/from GC or GD
import shutil

# For ploting
import seaborn as sns
import matplotlib.pyplot as plt

# For CatBoost modeling
!pip install catboost
from catboost import CatBoostClassifier

# For evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import roc_curve, auc

In [ ]:
# If saving and loading files in GDrive is needed
from google.colab import drive
drive.mount('/content/drive')

##**CatBoost model. Computation of new features**

##**1. Computation of new features: load and read csv files with prepocessed data  and masks, compute new features**

df1_ datasets should contain all the results after preprocessing ('PRICE_num', 'QUANTITY_num', 'HASHED_FRAUDRINGNAME_num', 'Label1').
GPU is needed

In [ ]:
# 1.1 Read the previously preprocessed data from csv files

df1_train_path = '...'
df1_train = pd.read_csv(df1_train_path, delimiter = ",", keep_default_na=False)

df1_val_path = '...'
df1_val = pd.read_csv(df1_val_path, delimiter = ",", keep_default_na=False)

df1_test_path = '...'
df1_test = pd.read_csv(df1_test_path, delimiter = ",", keep_default_na=False)


# Get new variable DAY

df1_train['DAY'] = pd.to_datetime(df1_train['_TRANSDATE']).dt.dayofweek
df1_val['DAY'] = pd.to_datetime(df1_val['_TRANSDATE']).dt.dayofweek
df1_test['DAY'] = pd.to_datetime(df1_test['_TRANSDATE']).dt.dayofweek

In [ ]:
# Before switching to RAPID/cudf.pandas it worth to compute masks on GPU (due to a bug with idmax we didn't understand).
# Alternativelly mask can be saved and loaded as csv files during RAPID session.

# 1.2 Function to create masks for indeces when:
# a) null values of PRICE and QUANTITY is droped
# b) labels of Non-Fraud/Fraud transactions are only for Fraud
# c) langauge of ITEM_NAMES is only English
# d) for carts defining by max-price item
# _mask_i - boolean mask for the a-c conditions (on items)
# _mask_c - boolean mask for the a-d conditions (on carts)

def masks():
  global train_mask_i
  global val_mask_i
  global test_mask_i
  global train_mask_c
  global val_mask_c
  global test_mask_c

  #train masks
  train_mask_i = (
    (~df1_train['PRICE_num'].isnull()) &
    (~df1_train['QUANTITY_num'].isnull()) &
    (df1_train['Label1'] != -1) &
    (df1_train['ENG'])
  )
  train_mask_c_idx = df1_train[train_mask_i].groupby('HASHED_SESSIONID')['PRICE_num'].idxmax()
  train_mask_c = pd.Series(False, index=df1_train.index)
  train_mask_c.iloc[train_mask_c_idx] = True

  #val masks
  val_mask_i = (
    (~df1_val['PRICE_num'].isnull()) &
    (~df1_val['QUANTITY_num'].isnull()) &
    (df1_val['Label1'] != -1) &
    (df1_val['ENG'])
  )
  val_mask_c_idx = df1_val[val_mask_i].groupby('HASHED_SESSIONID')['PRICE_num'].idxmax()
  val_mask_c = pd.Series(False, index=df1_val.index)
  val_mask_c.iloc[val_mask_c_idx] = True


  #test masks
  test_mask_i = (
    (~df1_test['PRICE_num'].isnull()) &
    (~df1_test['QUANTITY_num'].isnull()) &
    (df1_test['Label1'] != -1) &
    (df1_test['ENG'])
  )
  test_mask_c_idx = df1_test[test_mask_i].groupby('HASHED_SESSIONID')['PRICE_num'].idxmax()
  test_mask_c = pd.Series(False, index=df1_test.index)
  test_mask_c.iloc[test_mask_c_idx] = True

  return

masks()

In [ ]:
# The RAPID & cudf.pandas were used for boosting computation of new lag probabilistic features on dataframes
# It apeared that it worked quit fast and correct for some operations with pandas but for such one as idmax results were incorrect (maybe due to our errors in code)

# The next text and code in this cell from NVIDIA

# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py


In [ ]:
# For NVIDIA RAPIDS with cudf - to compute new features only
import cudf
#cudf.__version__
%load_ext cudf.pandas

In [ ]:
# 1.3 The function to calculate lag Probabilities P(ItemX|site_id)

def prob_day(df, dayX, item_name, site_id):
    #d0 = (dayX)  # Calculation for yesterday (standing at the point 23:59 of today)
    #d1 = (dayX - 1)  # Calculation for уesterday-1
    #d2 = (dayX - 2)  # Calculation for уesterday-2

  variables = {}

  for idx, d in enumerate(range(dayX, dayX-3, -1)):
    date_mask = (df['DAY'] == d)
    item_mask = (df['ITEM_NAME'] == item_name)
    site_mask = (df['HASHED_SITEID'] == site_id)

    mask_N_itemXsiteX = date_mask & item_mask & site_mask
    mask_N_siteX = date_mask & site_mask

    N_item_site = df.loc[mask_N_itemXsiteX]['QUANTITY_num'].sum()
    N_site = df.loc[mask_N_siteX]['QUANTITY_num'].sum()

    variables[f'P_siteX_day{idx}'] = N_item_site / N_site if N_site > 1e-6 else 0

  return variables['P_siteX_day0'], variables['P_siteX_day1'], variables['P_siteX_day2']

In [ ]:
# 1.4 the function to create new lag features in dataframes

def get_prob_features(df, df1_mask, class1ratio):
    if not all(col in df.columns for col in ['_TRANSDATE', 'ITEM_NAME', 'HASHED_SITEID', 'QUANTITY_num', 'Label1']):
        raise KeyError("One or more required columns are missing from the DataFrame")

    global downsampled_indices
    global mindate
    global df_downsampled
    global days

    df['DAY'] = pd.to_datetime(df['_TRANSDATE']).dt.day

    # Mask for downsampled the train set - to balance RF and FR
    class_1_indices = df[df1_mask].loc[df['Label1'] == 1].index  # RF
    class_2_indices = df[df1_mask].loc[df['Label1'] == 2].index  # FR
    np.random.seed(42)
    downsampled_class_1_indices = np.random.choice(class_1_indices, size=int(class1ratio*len(class_2_indices)), replace=False)
    downsampled_indices = np.concatenate([downsampled_class_1_indices, class_2_indices])

    mindate = min(df.loc[downsampled_indices, 'DAY']) + 2

    tqdm.pandas()

    # Extract relevant columns for downsampled indices
    df_downsampled = df.loc[downsampled_indices, ['DAY', 'ITEM_NAME', 'HASHED_SITEID']]

    # Select the day for each transaction date
    days = df_downsampled['DAY']

    # Initialize arrays to store probabilities
    P_siteX_day0 = np.zeros(len(df_downsampled))
    P_siteX_day1 = np.zeros(len(df_downsampled))
    P_siteX_day2 = np.zeros(len(df_downsampled))

    #breakpoint()

    # Iterate over the downsampled indices
    for i, (index, row) in enumerate(tqdm(df_downsampled.iterrows(), total=len(df_downsampled))):

        if row['DAY'] > mindate:
            P_siteX_day0[i], P_siteX_day1[i], P_siteX_day2[i] = prob_day(df, days.iloc[i], row['ITEM_NAME'], row['HASHED_SITEID'])

        else:
            P_siteX_day0[i], P_siteX_day1[i], P_siteX_day2[i] = np.nan, np.nan, np.nan

    # Assign the calculated probabilities back to the dataframe
    df.loc[downsampled_indices, 'P_siteX_day0'] = P_siteX_day0
    df.loc[downsampled_indices, 'P_siteX_day1'] = P_siteX_day1
    df.loc[downsampled_indices, 'P_siteX_day2'] = P_siteX_day2

    return df

In [ ]:
# 1.5 Get new features.

train_mask = train_mask_c
class1ratio = 100
get_prob_features(df1_train, train_mask, class1ratio)

val_mask = val_mask_c
class1ratio = 100
get_prob_features(df1_val,  val_mask, class1ratio)

test_mask = test_mask_c
class1ratio = 100
get_prob_features(df1_test,  test_mask, class1ratio)

In [ ]:
# Save and load results to GDrive


df_tosave = df1_train
prefix_name = 'df1_train_newfeatures'

filename = f'{prefix_name}.csv'
df_tosave.to_csv(filename, index=False)

source_path = f'/content/{filename}'
destination_path = f'/content/drive/My Drive/....../{filename}'

shutil.copy(source_path, destination_path)



##**2. Load csv files with df1, masks and new features, and npy datasets with embeddinds, concatinate the other variables**

In [ ]:
#2.1 Load df1 sets if they were saved on GDrive

In [ ]:
# 2.2 Load and read masks as csv files (if needed, no needs in case to train and predict fith P features - use 2.2 b))
# a) null values of PRICE and QUANTITY is droped
# b) labels of Non-Fraud/Fraud transactions are only for Fraud
# c) langauge of ITEM_NAMES is only English
# d) for carts defining by max-price item
# _mask_i - boolean mask for the a-c conditions (on items)
# _mask_c - boolean mask for the a-d conditions (on carts)

In [ ]:
# 2.2 b) Compute the masks to train and predict sets fith lag new features

train_mask_cnew = df1_train['P_siteX_day0'].notna() & df1_train['P_siteX_day1'].notna() & df1_train['P_siteX_day2'].notna()

val_mask_cnew = df1_val['P_siteX_day0'].notna() & df1_val['P_siteX_day1'].notna() & df1_val['P_siteX_day2'].notna()

test_mask_cnew = df1_test['P_siteX_day0'].notna() & df1_test['P_siteX_day1'].notna() & df1_test['P_siteX_day2'].notna()

In [ ]:
# 2.3 Load and read the saved npy files with embeddings for futher procesing

In [ ]:
# 2.4 Concatinate numpy datasets with 6 num features (including 3 lag features) and 3 cat features

# train_df

train_df = pd.DataFrame(train_embed, columns=[f'feature_{i}' for i in range(300)])
plusnumeric_df = df1_train[['PRICE_num', 'QUANTITY_num', 'RISK_SCORE', 'P_siteX_day0', 'P_siteX_day1', 'P_siteX_day2']]
categorical_df = df1_train[['HASHED_SITEID', 'PGROUP', 'DAY']]
train_df = pd.concat([train_df, plusnumeric_df, categorical_df], axis=1)
print(train_df.shape)

# val_df
val_df = pd.DataFrame(val_embed, columns=[f'feature_{i}' for i in range(300)])
plusnumeric_df = df1_val[['PRICE_num', 'QUANTITY_num', 'RISK_SCORE', 'P_siteX_day0', 'P_siteX_day1', 'P_siteX_day2']]
categorical_df = df1_val[['HASHED_SITEID', 'PGROUP', 'DAY']]
val_df = pd.concat([val_df, plusnumeric_df, categorical_df], axis=1)
print(val_df.shape)


#test_df
test_df = pd.DataFrame(test_embed, columns=[f'feature_{i}' for i in range(300)])
plusnumeric_df = df1_test[['PRICE_num', 'QUANTITY_num', 'RISK_SCORE', 'P_siteX_day0', 'P_siteX_day1', 'P_siteX_day2']]
categorical_df = df1_test[['HASHED_SITEID', 'PGROUP', 'DAY']]
test_df = pd.concat([test_df, plusnumeric_df, categorical_df], axis=1)
print(test_df.shape)


## **3. Functions to train and validate the CatBoost model**

In [ ]:
# 3.2 Function to train the CatBoost model

def catboost_train_val(params, class1ratio, train_mask, val_mask): #, test_mask):

    cat_features = params['cat_features']

    X_train = train_df.loc[train_mask]
    y_train = df1_train.loc[train_mask, 'Label1']
    X_val = val_df.loc[val_mask]
    y_val = df1_val.loc[val_mask, 'Label1']

    bst = CatBoostClassifier(**params)
    bst.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10)

    return bst


In [ ]:
# 3.4 Define parameters and train the model with masks taking into account only rows with the P features (P day0, P day1, P day2)

class1ratio = 100

train_mask = train_mask_cnew
val_mask   = val_mask_cnew

params = {
    'cat_features': ['HASHED_SITEID', 'PGROUP', 'DAY'],
    'iterations': 500,
    'learning_rate': 0.1,
    'depth': 6,
    'verbose': 10, # iteration for output 1 peace of information about training
    'scale_pos_weight': class1ratio,
}

bst = catboost_train_val(params, class1ratio, train_mask, val_mask)

In [ ]:
# 3.5 Predict with the trained model
test_mask  = test_mask_cnew
predicted_probabilities = bst.predict_proba(test_df[test_mask_cnew])

#pred = bst.predict(test_df[test_mask_cnew]) # binary label prediction

In [ ]:
# 3.6 Get and save features importance

feature_importances = bst.get_feature_importance()

importance_df = pd.DataFrame({
    'Feature': train_df.columns,
    'Importance': feature_importances
})
importance_df

importance_df.to_csv('feature_importances.csv', index=False)

In [ ]:
# 3.7 Evaluation of the trained model: trained with balance 100:1 and using only rows with existing values of lag features, changing the threshold of confidence

test_labels = df1_test[test_mask_cnew]['Label1'].values
test_labels = (test_labels == 2).astype(int)
preds_binary = (predicted_probabilities[:, 1] >= 0.9).astype(int)

accuracy = accuracy_score(test_labels, preds_binary)
precision = precision_score(test_labels, preds_binary)
recall = recall_score(test_labels, preds_binary)
f1 = f1_score(test_labels, preds_binary)
conf_matrix = confusion_matrix(test_labels, preds_binary)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

In [ ]:
# 3.8 ROC-AUC analysis

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(test_labels, predicted_probabilities[:, 1])

# Compute area under the curve (AUC)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:
# Save the binary label to df1_test and get metrics distribution by PGROUP
# Correct prediction of FR as FR - TP
# Wrong prediction of RF as FR   - FP
# Correct prediction of RF as RF - TN
# Wrong prediction of FR as RF   - FN

df1_test.loc[test_mask, 'FR_V1_100:1_309bin'] = preds_binary
df_test_filtered = df1_test[test_mask]

TP_mask = (df_test_filtered['Label1'] == 2) & (df_test_filtered['FR_V1_100:1_309bin'] == 1)
FP_mask = (df_test_filtered['Label1'] == 1) & (df_test_filtered['FR_V1_100:1_309bin'] == 1)
TN_mask = (df_test_filtered['Label1'] == 1) & (df_test_filtered['FR_V1_100:1_309bin'] == 0)
FN_mask = (df_test_filtered['Label1'] == 2) & (df_test_filtered['FR_V1_100:1_309bin'] == 0)

list_masks = {
    'TP': TP_mask,
    'FP': FP_mask,
    'TN': TN_mask,
    'FN': FN_mask
}

results = {mask_name: {} for mask_name in list_masks.keys()}
purchases = {mask_name: {} for mask_name in list_masks.keys()}

for vertical in df_test_filtered['PGROUP'].unique():
    for mask_name, mask in list_masks.items():
        num_cases = len(df_test_filtered[mask & (df_test_filtered['PGROUP'] == vertical)])
        item_names = df_test_filtered[mask & (df_test_filtered['PGROUP'] == vertical)].groupby(['HASHED_SITEID'])['ITEM_NAME'].apply(list).reset_index()
        print(item_names)

        results[mask_name][vertical] = num_cases
        purchases[mask_name][vertical] = item_names

df_errors_vertical    = pd.DataFrame(results)
#df_purchases_vertical = pd.DataFrame(purchases)

df_errors_vertical['Precision'] = df_errors_vertical.apply(
    lambda row: row['TP'] / (row['TP'] + row['FP']) if (row['TP'] + row['FP']) > 0 else 0,
    axis=1
)
df_errors_vertical['Recall'] = df_errors_vertical.apply(
    lambda row: row['TP'] / (row['TP'] + row['FN']) if (row['TP'] + row['FN']) > 0 else 0,
    axis=1
)
new_columns_order = ['Precision', 'Recall'] + [col for col in df_errors_vertical.columns if col not in ['Precision', 'Recall']]
df_errors_vertical = df_errors_vertical[new_columns_order]

df_errors_vertical.to_csv('df_errors_FR.xlsx', index=True)

purchases